In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy
import io
from google.colab import drive
import threading
import time
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import glob
glob.glob("/content/drive/My Drive/google_notebook/M5_traindata2/*")

['/content/drive/My Drive/google_notebook/M5_traindata2/calendar2.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/calendar.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/sales_train_validation.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/sample_submission.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/sell_prices.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/sell_prices2.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a']

In [0]:
import os.path

train_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/sales_train_validation.csv")
calendar_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/calendar.csv")
sellprices2_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/sell_prices2.csv")
calendar_data2 = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/calendar2.csv")

days_num = 1913

cols = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id", "wm_yr_wk", "wday", "month", "year", "d",
        "Religious", "National", "Cultural", "Sporting", "snap", "price", "price_down", "price_up", "value"]
train_data2a0 = pd.DataFrame(index=[ij for ij in range(days_num)], columns=cols)

start_n = 0

thread_num = 5 #スレッド数

print(len(train_data))

current_num = [-1 for jj in range(thread_num)]  # 実行中のファイル番号(重複処理を防ぐ)


class traindata2Threading(threading.Thread):

    def __init__(self, thread_name):
        self.thread_name = str(thread_name)
        self.thread_number = int(thread_name)
        threading.Thread.__init__(self)

    def __str__(self):
        return self.thread_name

    def run(self):
        for ii in range(start_n, len(train_data)):
            if ((ii in current_num) == False): #その行は処理されていない
                current_num[self.thread_number] = ii
                if (os.path.isfile("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv") == False):  # ファイルがまだ作成されていない
                    train_data2a = train_data2a0.copy()
                    tra_sir = train_data.iloc[ii]
                    for jj in range(days_num):train_data2a.iloc[jj,0:6] = tra_sir[0:6]
                    train_data2a.iloc[:,6] = calendar_data.iloc[:,1]
                    train_data2a.iloc[:,7:11] = calendar_data.iloc[:,3:7]
                    train_data2a.iloc[:,11:15] = calendar_data2.iloc[:,1:5]
                    train_data2a.iloc[:,16] = self.stap(tra_sir[])


                    train_data2a.to_csv(
                        "/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv")
                current_num[self.thread_number] = -1
                print('\r', end='')
                print(current_num, end='')

thread_list = []
for i in range(thread_num):
    thread = traindata2Threading(thread_name=i)
    thread.start()
    thread_list.append(thread)
    sleep_seconds = 1
    time.sleep(sleep_seconds)

for thread in thread_list:
    thread.join()

In [4]:
cols = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id", "wm_yr_wk", "wday", "month", "year", "d",
        "Religious", "National", "Cultural", "Sporting", "snap", "price", "price_down", "price_up", "value"]
train_data2a0 = pd.DataFrame(index=[ij for ij in range(1913)], columns=cols)
train_data2a0

,id,item_id,dept_id,cat_id,store_id,state_id,wm_yr_wk,wday,month,year,d,Religious,National,Cultural,Sporting,snap,price,price_down,price_up,value
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
calendar_data2 = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/calendar2.csv")
calendar_data2

,Unnamed: 0,d,Religious,National,Cultural,Sporting,snap_CA,snap_TX,snap_WI
0,0,d_1,0,0,0,0,0,0,0
1,1,d_2,0,0,0,0,1,1,0
2,2,d_3,0,0,0,0,1,1,1
3,3,d_4,0,0,0,0,2,2,1
4,4,d_5,0,0,0,0,2,1,2
...,...,...,...,...,...,...,...,...,...
1964,1964,d_1965,0,0,0,0,0,2,2
1965,1965,d_1966,0,0,0,0,0,1,1
1966,1966,d_1967,0,0,1,1,0,1,1
1967,1967,d_1968,0,0,1,1,0,0,0
